# Algorithmes de LZ78 et LZSS

## 1. Huffman

Dans cette partie, on remet les fonctions permettant de réaliser la compression par la méthode de Huffman. Vous pouvez garder celles proposées ci-dessous, ou mettre les votre (ou encore mieux, appeler votre package pour huffman).

Il faut également avoir une fonction ``huffmanSize(text)`` qui retourne la taille, en nombre de bits, de la compression du message ``text`` au moyen de la méthode de Huffman.

In [1]:
def count(L):
    D = {}
    for a in L:
        if a not in D:
            D[a] = 1
        else:
            D[a] += 1
    return [(v, k) for (k, v) in D.items()]


class Heap:
    def __init__(self, function=lambda x: x):
        self.tab = []
        self.f = function

    def father(self, i):
        if i == 0:
            return None
        return int((i - 1) / 2)

    def left(self, i):
        if 2 * i + 1 < len(self.tab):
            return 2 * i + 1
        return None

    def right(self, i):
        if 2 * i + 2 < len(self.tab):
            return 2 * i + 2
        return None

    def __str__(self):
        return str(self.tab)

    def add(self, x):
        self.tab.append(x)
        i = len(self.tab) - 1
        while i > 0 and self.f(self.tab[i]) < self.f(self.tab[self.father(i)]):
            self.tab[i], self.tab[self.father(i)] = self.tab[self.father(i)], self.tab[i]
            i = self.father(i)

    def extract(self):
        if len(self.tab) == 1:
            return self.tab.pop()
        res = self.tab[0]
        self.tab[0] = self.tab.pop()
        i = 0
        while True:
            l, r = self.left(i), self.right(i)
            vl, vr = self.f(self.tab[i]), self.f(self.tab[i])
            if l is not None:
                vl = self.f(self.tab[l])
            if r is not None:
                vr = self.f(self.tab[r])
            if self.f(self.tab[i]) <= vr and self.f(self.tab[i]) <= vl:
                return res
            if vl < vr:
                self.tab[i], self.tab[l] = self.tab[l], self.tab[i]
                i = l
            else:
                self.tab[i], self.tab[r] = self.tab[r], self.tab[i]
                i = r


def huffmanTree(L):
    C = count(L)
    H = Heap(lambda x: x[0])
    for c in C:
        H.add(c)
    while len(H.tab) > 1:
        x1 = H.extract()
        x2 = H.extract()
        A = (x1[0] + x2[0], x1, x2)
        H.add(A)
    return H.tab[0]


def huffmanToCode(T, C=None, word=None):
    if word is None:
        word = []
    if C is None:
        C = {}
    if len(T) == 2:  # leaf
        C[T[1]] = u"".join(word)
    else:
        word.append('0')
        huffmanToCode(T[1], C, word)
        word[-1] = '1'
        huffmanToCode(T[2], C, word)
        word.pop()
    return C


from math import log


def huffmanSize(text):
    '''retourne la taille (en nombre de bits) une fois compressé par Huffman'''
    K = count(text)
    T = huffmanTree(text)
    C = huffmanToCode(T)
    k = len(C)
    b = int(log(k + 1, 2) + 1)
    taille_compress = sum([len(C[l]) * nb for nb, l in K])
    taille_arbre = (b + 2) * k - 1
    return taille_arbre + taille_compress

## 2. LZ78

Implantez l'algorithme de LZ78. La fonction aura pour signature ``lz78(u, size=None)``, où ``size`` représente la taille maximale du dictionnaire (voir le cours). Si ``size`` vaut ``None``, alors le dictionnaire n'a pas de limite de taille. Sinon, il est vidé dès qu'il atteint cette taille (voir le cours), et réinitialisé pour ne plus contenir que le mot vide.

La sortie de l'algorithme sera, comme dans le cours, une liste de paires (indice,caractère).

In [2]:
def lz78(u, size=None):
    dico = {'': 0}
    lst = []
    i = 1
    tmp = ''
    for lettre in u:
        if tmp + lettre not in dico:
            lst.append((dico[tmp], lettre))  #ajout à la liste de résultat

            if (size != None and i >= size):  #Si on fixe une taille au dico et qu'elle est pleine
                dico = {'': 0}
                i = 1

            dico[tmp + lettre] = i  #ajout dico
            tmp = ''
            i += 1
        else:
            tmp += lettre
    if tmp not in '':
        lst.append((dico[tmp[:-1]], tmp[-1]))
    return lst


Vérifiez que votre fonction fonctionne correctement avec l'exemple du cours.

In [3]:
mot = "ccbbacbaabcbbcbcccaa"
print(lz78(mot))
print(lz78(mot, 5))

[(0, 'c'), (1, 'b'), (0, 'b'), (0, 'a'), (2, 'a'), (4, 'b'), (2, 'b'), (2, 'c'), (1, 'c'), (4, 'a')]
[(0, 'c'), (1, 'b'), (0, 'b'), (0, 'a'), (2, 'a'), (0, 'a'), (0, 'b'), (0, 'c'), (3, 'b'), (0, 'c'), (0, 'b'), (2, 'c'), (2, 'a'), (0, 'a')]


Ecrire la fonction `un_lz78(seq,size=None)` qui décompresse la séquence `seq` produite par `lz78`.

In [4]:
def un_lz78(seq, size=None):
    dico = {}
    mot = ""
    i = 1
    for (u, v) in seq:
        if (u not in dico):
            if (size != None and i >= size):  #Si on fixe une taille au dico et qu'elle est pleine
                dico = {}
                i = 1
            mot += v
            dico[i] = v
        else:
            tmp = dico[u] + v
            mot += tmp
            if (size != None and i >= size):  #Si on fixe une taille au dico et qu'elle est pleine
                dico = {}
                i = 1
            dico[i] = tmp
        i += 1
    return mot


print(un_lz78(lz78(mot, 4), 4))

ccbbacbaabcbbcbcccaa


Vérifiez que votre fonction fonctionne avec le script ci-dessous.

In [5]:
from random import choice

for _ in range(10):
    u = "".join([choice(['a', 'b', 'c']) for _ in range(20)])
    print(u, un_lz78(lz78(u)) == u)

bacbbcbbacbbcacbcbcc True
cccabcabaaaaaccbbaaa True
abbbbabbccbcccbaaccb True
cccbcbacbcaabbabbbac True
abbcaccacacbbcccaaca True
bbabacbccabcacbbaaba True
ccacbbcacabcaacabccc True
acaaacaaaabacbacbcbc True
bbaccbcbaaccbbabbcab True
cbacaacaacbbbacbabbc True


## 3. Tests LZ78 + Huffman

Appliquez lz78 au fichier `etranger.txt` (en ouvrant le fichier avec l'option 'r', pas 'rb'). Quelle est la taille de la compression du résultat de lz78 par Huffman (sans limite de taille pour l'alphabet)? Comparez-là à la taille initiale du fichier `etranger.txt`.

In [6]:
fichier = open("etranger.txt", "r")
texte = fichier.read()
lst = lz78(texte)
size = huffmanSize(lst)
print("La taille de la compression du résultat de lz78 par Huffman est de ", size, " bits.")

La taille de la compression du résultat de lz78 par Huffman est de  1141348  bits.


On souhaite maintenant utiliser deux arbres de Huffman : un pour les indices, et un pour les caractères. Quelle deviendrait la taille du fichier compressé si on appliquait cette technique à `etranger.txt` ?

In [7]:
a = ""
b = ""
for (u, v) in lst:
    a += str(u)
    b += v
size = huffmanSize(a) + huffmanSize(b)
print("La taille du fichier compressé si on appliquait cette technique à etranger.txt est de ", size, " bits.")

La taille du fichier compressé si on appliquait cette technique à etranger.txt est de  640152  bits.


Même question si on limite la taille du dictionnaire à 2048.

In [8]:
lst = lz78(texte, 2048)
a = ""
b = ""
for (u, v) in lst:
    a += str(u)
    b += v
size = huffmanSize(a) + huffmanSize(b)
print("La taille du fichier compressé si on appliquait cette technique à etranger.txt est de ", size, " bits.")

La taille du fichier compressé si on appliquait cette technique à etranger.txt est de  745118  bits.


## 4. LZW (Facultatif)

Reprendre les parties 2. et 3. avec LZW au lieu de LZ78.

In [9]:
import copy


def lzw(u, size=None):
    alpha = {m for m in u}
    i = 0
    dico = {"": -1}
    tmp = ""
    tmp2 = ""
    lst = []
    for x in sorted(alpha):
        dico[x] = i
        i += 1
    dicoCopy = copy.deepcopy(dico)

    for lettre in u:
        if tmp not in dico:  #Ajout dans le dico
            if (size != None and i >= size):  #Si on fixe une taille au dico et qu'elle est pleine
                dico = copy.deepcopy(dicoCopy)
                i = len(dico)
            dico[tmp] = i
            i += 1
            tmp = tmp[-1]
        tmp += lettre

        if tmp2 + lettre not in dico:  #Ajout dans la liste de resultat
            lst.append(dico[tmp2])
            tmp2 = lettre
        else:
            tmp2 += lettre

    lst.append(dico[tmp2])
    return lst

In [10]:
mot = "ccbbacbaabcbbcbcccaa"
print(lzw(mot))
print(lzw(mot, 6))

[2, 2, 1, 1, 0, 4, 0, 0, 1, 4, 11, 11, 3, 9]
[2, 2, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 1, 2, 1, 2, 4, 0, 0]


In [11]:
def un_lzw(seq, dico, size=None):
    mot = ""
    tmp = ""
    i = len(dico)
    dicoCopy = copy.deepcopy(dico)
    for x in seq:
        if x not in dico:
            tmpn = tmp
            itmp = 0
            arret = False
            while True:
                if arret:
                    break
                tmp += tmp[itmp]
                itmp += 1
                if tmp not in dico.values():
                    if (size != None and i >= size):  #Si on fixe une taille au dico et qu'elle est pleine
                        dico = copy.deepcopy(dicoCopy)
                        i = len(dico)
                    dico[i] = tmp
                    mot += dico[i]
                    i += 1
                    arret = True
        else:
            for y in dico[x]:
                tmp += y
                mot += y
                if tmp not in dico.values():
                    if (size != None and i >= size):  #Si on fixe une taille au dico et qu'elle est pleines
                        dico = copy.deepcopy(dicoCopy)
                        i = len(dico)
                    dico[i] = tmp
                    tmp = tmp[-1]
                    i += 1
    return mot

In [12]:
print(lzw("ccbbacbaabcbbcbcccaa"))
print(un_lzw([2, 2, 1, 1, 0, 4, 0, 0, 1, 4, 11, 11, 3, 9], {0: "a", 1: "b", 2: "c"}))
print(un_lzw([2, 2, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 1, 2, 1, 2, 4, 0, 0], {0: "a", 1: "b", 2: "c"}, 6))

[2, 2, 1, 1, 0, 4, 0, 0, 1, 4, 11, 11, 3, 9]
ccbbacbaabcbbcbcccaa
ccbbacbaabcbbcbcccaa


In [13]:
from random import choice

for _ in range(10):
    u = "".join([choice(['a', 'b', 'c']) for _ in range(20)])
    print(u, un_lzw(lzw(u), {0: "a", 1: "b", 2: "c"}) == u)

baccbababccbaaccbbbc True
ccabcbbabbcaaacbabba True
cabaacccaccaccccaaac True
ababbbaaabccbbbaccbb True
cbacbbbbabcabbabbcba True
bbbabbcbababcccbcacb True
cacbcbababcabcaabcba True
cccaccbbabaaababcaac True
abaacbcbaaccabaabcbc True
ccabcacabbacbabacbcc True


In [14]:
#Partie 3 : LZW + Huffman

In [15]:
fichier = open("etranger.txt", "r")
texte = fichier.read()
lst = lzw(texte)
size = huffmanSize(lst)
print("La taille du fichier compressé si on appliquait cette technique à etranger.txt est de ", size, " bits.")

La taille du fichier compressé si on appliquait cette technique à etranger.txt est de  892972  bits.


In [16]:
#2e compression avec 2048

In [17]:
lst = lzw(texte, 2048)
size = huffmanSize(lst)
print("La taille du fichier compressé si on appliquait cette technique à etranger.txt est de ", size, " bits.")

La taille du fichier compressé si on appliquait cette technique à etranger.txt est de  792196  bits.
